# Prompt Chaining and Sequencing Tutorial

## Overview

This tutorial delves into the powerful techniques of prompt chaining and sequencing when working with large language models. We'll demonstrate these concepts using Google's Gemini via OpenRouter and LangChain, showing you how to create sophisticated, multi-step AI-driven processes.

## Motivation

As AI applications grow in complexity, it becomes crucial to break down intricate tasks into manageable steps. Prompt chaining and sequencing provide a framework for guiding language models through a series of interconnected prompts, resulting in more structured and controlled outputs. This approach is invaluable for tasks requiring multiple stages of processing or decision-making.

## Key Components

1. **Basic Prompt Chaining**: Linking outputs from one prompt to inputs of another.
2. **Sequential Prompting**: Designing a logical progression of prompts to guide the AI through multi-step processes.
3. **Dynamic Prompt Generation**: Utilizing the output of one prompt to adaptively create the next prompt.
4. **Error Handling and Validation**: Implementing safeguards and quality checks within the prompt chain.

## Setup

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
import re
from os import getenv

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="google/gemini-flash-1.5",
)

## Basic Prompt Chaining

Let's start with a simple example of prompt chaining. We'll create two prompts: one to generate a short story, and another to summarize it.

In [2]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"], template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}",
)

# Chain the prompts


def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary


# Test the chain
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: The crimson sun bled across Xylos, casting long shadows from the bioluminescent fungi forests.  Elara, her cybernetic arm humming faintly, crept through the undergrowth, her mission to retrieve the stolen stardust crystals before the Kryll Swarm reached them.  Failure meant not only the death of her people, but the extinction of all life on Xylos.  The crystals pulsed with a faint, ethereal light, a beacon and a deadly threat.


Summary: On the dying planet Xylos, Elara raced against a deadly swarm to retrieve stolen stardust crystals, the fate of her people and all life hanging in the balance.



## Sequential Prompting

Now, let's create a more complex sequence of prompts for a multi-step analysis task. We'll analyze a given text for its main theme, tone, and key takeaways.

In [3]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}",
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}",
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}",
)


def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (
        (takeaway_prompt | llm)
        .invoke({"text": text, "theme": theme, "tone": tone})
        .content
    )
    return {"theme": theme, "tone": tone, "takeaways": takeaways}


# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme is the **dual nature of artificial intelligence and the need for responsible development**.  The text highlights both the potential benefits and the potential risks of AI, emphasizing the importance of careful consideration and proactive measures to mitigate its negative consequences.


Tone: The overall tone of the text is **cautiously optimistic**.  While acknowledging the potential benefits and revolutionary aspects of AI, it emphasizes the need for careful consideration of the associated risks and ethical implications.  It's not purely positive nor purely negative, but rather a balanced perspective that leans towards a responsible and measured approach to AI's development.


Takeaways: The key takeaways from the text are:

* **AI's Dual Nature:** Artificial intelligence presents both immense potential benefits (revolutionizing industries, improving daily life) and significant risks (privacy concerns, job displacement, potential for misuse).  This duality is ce

## Dynamic Prompt Generation

In this section, we'll create a dynamic question-answering system that generates follow-up questions based on previous answers.

In [4]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}",
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question.",
)


def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})

        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (
                (follow_up_prompt | llm)
                .invoke({"question": current_question, "answer": answer})
                .content
            )

    return qa_chain


# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Drug discovery, materials science, financial modeling, cryptography, optimization problems, and artificial intelligence.


Q2: What are the biggest current challenges hindering the wider application of quantum computing in these fields?

A2: High cost, qubit fragility, limited scalability, and lack of error correction are the biggest current challenges hindering wider application of quantum computing.


Q3: Given the limitations of high cost, qubit fragility, limited scalability, and lack of error correction, what are the most promising research avenues currently being pursued to overcome these challenges and accelerate the adoption of quantum computing?

A3: Promising avenues include developing more stable qubits (e.g., topological qubits), improving fabrication techniques for scalability (e.g., silicon-based qubits), and advancing error correction codes and fault-tolerant architectures.


Q4: Given the diverse approach

## Error Handling and Validation

In this final section, we'll implement error handling and validation in our prompt chains to make them more robust.

In [5]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text.",
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why.",
)


def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r"\b\d{4}\b", text)
    return match.group() if match else None


def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)

            if not number:
                raise ValueError(
                    f"Failed to extract a 4-digit number from the response: {response}"
                )

            # Validate the relevance
            validation = (
                (validate_prompt | llm)
                .invoke({"number": number, "topic": topic})
                .content
            )
            if validation.lower().startswith("yes"):
                return number
            else:
                print(
                    f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}"
                )
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")

    return "Failed to generate a valid number after multiple attempts."


# Test the robust number generation
topic = "Điện Biên Phủ"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'Điện Biên Phủ': 1954
